In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fastai.core import *

%matplotlib notebook

In [2]:
! ls -R 

.:
data  MRNet_EDA.ipynb

./data:
train		    train-acl.csv	valid		    valid-acl.csv
train-abnormal.csv  train-meniscus.csv	valid-abnormal.csv  valid-meniscus.csv

./data/train:
axial  coronal	sagittal

./data/train/axial:
0000.npy  0142.npy  0284.npy  0426.npy	0568.npy  0710.npy  0852.npy  0994.npy
0001.npy  0143.npy  0285.npy  0427.npy	0569.npy  0711.npy  0853.npy  0995.npy
0002.npy  0144.npy  0286.npy  0428.npy	0570.npy  0712.npy  0854.npy  0996.npy
0003.npy  0145.npy  0287.npy  0429.npy	0571.npy  0713.npy  0855.npy  0997.npy
0004.npy  0146.npy  0288.npy  0430.npy	0572.npy  0714.npy  0856.npy  0998.npy
0005.npy  0147.npy  0289.npy  0431.npy	0573.npy  0715.npy  0857.npy  0999.npy
0006.npy  0148.npy  0290.npy  0432.npy	0574.npy  0716.npy  0858.npy  1000.npy
0007.npy  0149.npy  0291.npy  0433.npy	0575.npy  0717.npy  0859.npy  1001.npy
0008.npy  0150.npy  0292.npy  0434.npy	0576.npy  0718.npy  0860.npy  1002.npy
0009.npy  0151.npy  0293.npy  0435.npy	0577.npy  0719.npy  0861.npy  1003.np

In [3]:
data_path = Path('./data')
train_path = data_path/'train'
valid_path = data_path/'valid'

In [4]:
train_abnl = pd.read_csv(data_path/'train-abnormal.csv', header=None,
                       names=['Case', 'Abnormal'], 
                       dtype={'Case': str, 'Abnormal': np.int64})
print(train_abnl.groupby('Abnormal').count())
train_abnl.head()

          Case
Abnormal      
0          217
1          913


,Case,Abnormal
0,0000,1
1,0001,1
2,0002,1
3,0003,1
4,0004,1


In [5]:
train_acl = pd.read_csv(data_path/'train-acl.csv', header=None,
                       names=['Case', 'ACL_tear'], 
                       dtype={'Case': str, 'ACL_tear': np.int64})
print(train_acl.groupby('ACL_tear').count())
train_acl.head()

          Case
ACL_tear      
0          922
1          208


,Case,ACL_tear
0,0000,0
1,0001,1
2,0002,0
3,0003,0
4,0004,0


In [6]:
train_meniscus = pd.read_csv(data_path/'train-meniscus.csv', header=None,
                       names=['Case', 'Meniscus_tear'], 
                       dtype={'Case': str, 'Meniscus_tear': np.int64})
print(train_meniscus.groupby('Meniscus_tear').count())
train_meniscus.head()

               Case
Meniscus_tear      
0               733
1               397


,Case,Meniscus_tear
0,0000,0
1,0001,1
2,0002,0
3,0003,1
4,0004,0


In [7]:
def load_one_stack(case, data_path=train_path, plane='coronal'):
    fpath = data_path/plane/'{}.npy'.format(case)
    return np.load(fpath)

def load_stacks(case):
    x = {}
    planes = ['axial', 'coronal', 'sagittal']
    for i, plane in enumerate(planes):
        x[plane] = load_one_stack(case, plane=plane)
    return x

In [8]:
case = train_abnl.Case[0]
x = load_one_stack(case)
print(x.shape)
print(x.max())

(36, 256, 256)
255


In [9]:
x_multi = load_stacks(case)
x_multi

{'axial': array([[[ 0,  0,  0,  0, ...,  4,  5,  4,  3],
         [ 0,  0,  0,  0, ...,  8,  8,  6,  8],
         [ 0,  0,  0,  0, ..., 14, 14, 11, 11],
         [ 0,  0,  0,  0, ..., 16, 16, 14, 15],
         ...,
         [ 0,  0,  0,  0, ..., 14, 15, 18, 16],
         [ 0,  0,  0,  0, ..., 15, 16, 15, 12],
         [ 0,  0,  0,  0, ..., 11, 12, 13, 12],
         [ 0,  0,  0,  0, ...,  8, 11,  7,  9]],
 
        [[ 0,  0,  0,  0, ...,  4,  3,  2,  2],
         [ 0,  0,  0,  0, ...,  5,  9,  7,  7],
         [ 0,  0,  0,  0, ..., 10, 13, 10, 10],
         [ 0,  0,  0,  0, ..., 14, 14, 19, 17],
         ...,
         [ 0,  0,  0,  0, ..., 18, 16, 16, 17],
         [ 0,  0,  0,  0, ..., 13, 12, 15, 13],
         [ 0,  0,  0,  0, ..., 16, 14, 12, 12],
         [ 0,  0,  0,  0, ...,  8,  6,  5,  7]],
 
        [[ 0,  0,  0,  0, ...,  1,  1,  1,  1],
         [ 0,  0,  0,  0, ...,  7,  8,  6,  6],
         [ 0,  0,  0,  0, ..., 12, 11, 13, 10],
         [ 0,  0,  0,  0, ..., 12, 18, 18, 16

In [10]:
from ipywidgets import interactive
from IPython.display import display

plt.style.use('grayscale')

class KneePlot():
    def __init__(self, x, figsize=(10, 10)):
        self.x = x
        self.slice_range = (0, self.x.shape[0] - 1)
        self.resize(figsize)
    
    def _plot_slice(self, im_slice):
        fig, ax = plt.subplots(1, 1, figsize=self.figsize)
        ax.imshow(self.x[im_slice, :, :])
        plt.show()

    def resize(self, figsize):
        self.figsize = figsize
        self.interactive_plot = interactive(self._plot_slice, im_slice=self.slice_range)
        self.output = self.interactive_plot.children[-1]
        self.output.layout.height = '{}px'.format(60 * self.figsize[1])

    def show(self):
        display(self.interactive_plot)


In [11]:
plot = KneePlot(x)
plot.show()


interactive(children=(IntSlider(value=17, description='im_slice', max=35), Output(layout=Layout(height='600px'…

In [12]:
plot.resize(figsize=(12, 12))
plot.show()


interactive(children=(IntSlider(value=17, description='im_slice', max=35), Output(layout=Layout(height='720px'…

In [15]:
from ipywidgets import interact, Dropdown, IntSlider

class MultiKneePlot():
    def __init__(self, x_multi, figsize=(10, 10)):
        self.x = x_multi
        self.planes = ['coronal', 'sagittal', 'axial']
        self.slice_nums = {plane: self.x[plane].shape[0] for plane in self.planes}
        self.figsize = figsize
    
    def _plot_slices(self, plane, im_slice): 
        fig, ax = plt.subplots(1, 1, figsize=self.figsize)
        ax.imshow(self.x[plane][im_slice, :, :])
        plt.show()
    
    def draw(self):
        planes_widget = Dropdown(options=self.planes)
        plane_init = self.planes[0]
        slice_init = self.slice_nums[plane_init] - 1
        slices_widget = IntSlider(min=0, max=slice_init, value=slice_init//2)
        def update_slices_widget(*args):
            slices_widget.max = self.slice_nums[planes_widget.value] - 1
            slices_widget.value = slices_widget.max // 2
        planes_widget.observe(update_slices_widget, 'value')
        interact(self._plot_slices, plane=planes_widget, im_slice=slices_widget)
    
    def resize(self, figsize): self.figsize = figsize


In [16]:
plot_multi = MultiKneePlot(x_multi)
plot_multi.draw()

interactive(children=(Dropdown(description='plane', options=('coronal', 'sagittal', 'axial'), value='coronal')…